In [ ]:
# import dependencies
import pandas as pd
import requests
from bs4 import BeautifulSoup
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import re

In [ ]:
# set the chromedriver path
executable_path = {'executable_path':'chromedriver.exe'}
browser = Browser("chrome", **executable_path, headless=False)

## Scaraping the data

In [ ]:
locIds = ["1133904","1151614","1140171","1150505","1147401","1139761","1147436","1154532","1138213","1152672"]
base_url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId="

city_urls = []

for locid in locIds:
     city_url = base_url + locid
     city_urls.append(city_url)

In [ ]:
city_url

In [ ]:
browser.visit(city_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
companies = []
ratings = []
salaries = []
cities = []
states = []
job_descriptions = []
xpath = "//*[@id='MainCol']/div/ul/li"
def save_results():
    results = browser.find_by_xpath(xpath)
    for res in results:
        try:
            salaries.append(res.find_by_css("div:nth-child(3) > div:nth-child(1) > span").text)
            company = res.find_by_css("div.flexbox.empLoc > div").text
            companies.append(company.split('–')[0])
            cities.append(company.split('–')[1])
            states.append(company.split(',')[1])
            ratings.append(res.find_by_css("div.logoWrap > span").text)
            job_descriptions.append(browser.find_by_id("JobDescriptionContainer").text)
        except:
            pass

def close_modal():
    if len(browser.find_by_css(".xBtn")):
        browser.find_by_css(".xBtn")[0].click()

def is_last_page():
    if len(browser.find_by_css("li.page.current.last")):
        return True
    return False

def next_page():
    next_pages = browser.find_by_css("li.next")
    next_pages[0].click()

def crawl_city(city_url):
    print(city_url)
    browser.visit(city_url)
    last_page = False
    counter = 0
    while not last_page:
        if is_last_page(): last_page = True
        close_modal()
        print(f"Processing Page {counter+1}")
        save_results()
        counter += 1
        next_page()

In [ ]:
companies

In [ ]:
df = pd.DataFrame({"Companies": companies, 
                   "Salaries": salaries, 
                   "Ratings": ratings,
                   "Job_descriptions": job_descriptions
                  })

In [ ]:
phoenix_url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1133904&jobType="
portland_url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1151614&jobType="
houston_url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1140171&jobType="
seattle_url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1150505&jobType="
sanfrancisco_url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1147401&jobType="
austin_url = "https://www.glassdoor.com/Job/jobs.htm?sc.keyword=Data%20Scientist&locT=C&locId=1139761&locKeyword=Austin, %20TX&srs=RECENT_SEARCHES"
sanjose_url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1147436&jobType="
boston_url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1154532&jobType="
washington_url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1138213&jobType="
philadelphia_url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1152672&jobType="



In [ ]:
city_url = [philadelphia_url, phoenix_url,houston_url,seattle_url,sanfrancisco_url,sanjose_url,austin_url,boston_url,washington_url,portland_url]